Prepare a file to generate the [`Wordle`](https://gher-ulg.github.io/Liege-Colloquium/topicwordle.html) using the list of topics (file [topic.md](https://github.com/gher-ulg/gher-ulg.github.io/blob/master/Liege-Colloquium/topics.md)).

In [1]:
import os
import re
import operator
from collections import Counter

File to read titles from:

In [2]:
datafile = "/home/ctroupin/ULg/gher-ulg.github.io/Liege-Colloquium/topics.md"
jsonfile = "../data/topicsWordle.js"
jsonfile2 = "/home/ctroupin/ULg/gher-ulg.github.io/data/topicsWordle.js"

In [3]:
topiclist = []
with open(datafile, "r") as f:
    for lines in f:
        # Split line with | as separator
        topic = lines.split("|")[2]
        # Convert to lower case
        topic = [x.lower() for x in topic.split()]
        # Remove final 's' (assuming it's plural)
        # topic = [x.rstrip()[:-1] if (x.rstrip()[-1] == "s") else x.rstrip() for x in topic]
        # Add to list
        topiclist.extend(topic)
# Convert list to string
wordstring = " ".join(topiclist)

## Remove punctuation and common words

In [4]:
punctuation = [":", ".", "–", '"', ",", "'"]

removelist = ["\n", "the", "in", "a", "of", "les",
              "from" "le", "à", "but",
              "and","to", "for", "at", "on", 
              "no", "specific", "topic", "identifiable", "contents",
              "several", "papers", "related", "easily"
              "contents"]

substdict = {"l’océan": "ocean",
              "from": " ",
              "équations": "equations",
              "mesoscale/synoptic": "mesoscale synoptic",
              "modeling": "modelling", "no specific topic": " ",
              "sub-mesoscale": "submesoscale", 
              "hydrodynamically": 'hydrodynamics', 
              "hydrodynamical": 'hydrodynamics', 
              "hydrodynamiques": 'hydrodynamics', 
              "hydrodynamic": 'hydrodynamics', 
              "seas": "sea", "changes": "change",
              "waters": "water", 
              "ecosystems": "ecosystem",
              "estuarine": "estuaries",
              "applicable": "applicable",
              "tracers": "tracer",
              "models": "modelling",
              "re³-visited": "revisited",
              "re-revisited": "revisited",
              "environmental": "environments",
              "remote sensing": "remote-sensing",
              "bio-geo-chemical": "biogeochemical",
              "analyses": "analysis",
              "interaction": "interactions",
              "prediction": "predictions"
               }

for symbols in punctuation:
    wordstring = wordstring.replace(symbols, ' ')
for words in removelist:
    wordstring = re.sub(r"\b{}\b".format(words), ' ', wordstring)
for oldvalue, newvalue in substdict.items():
    wordstring = re.sub(r"{}\b".format(oldvalue), newvalue, wordstring)

## Count word frequency and sort

In [5]:
worddict = Counter(wordstring.split())
sortedworddict = sorted(worddict.items(), key=operator.itemgetter(1), reverse=True)

# Convert the dictionary to json
We should get something like this: 
```
var words = [
  {text: 'have', size: 102},
  {text: 'Oliver', size: 47},
  {text: 'say', size: 46},
  {text: 'said', size: 36}
]
```

In [6]:
with open(jsonfile, "w") as jf:
    jf.write("var words = [\n")
    for lines in sortedworddict:
        jf.write("".join(("\t{text: '", lines[0], "', size: ", str(lines[1]), "},\n")))
    jf.write("]")

# Build dictionary with the years for each word

In [8]:
clqdict = {}
with open(datafile, "r") as f:
    for lines in f:
        # Split line with | as separator
        year = int(lines.split("|")[1])
        topic = lines.split("|")[2]
        
        for wordstring in topic.split():
            # Convert to lower case
            wordstring = wordstring.lower()
            
            for symbols in punctuation:
                wordstring = wordstring.replace(symbols, ' ')
            for words in removelist:
                wordstring = re.sub(r"\b{}\b".format(words), ' ', wordstring)
            for oldvalue, newvalue in substdict.items():
                wordstring = re.sub(r"{}\b".format(oldvalue), newvalue, wordstring)
            
            for w in wordstring.split():
                if w not in removelist:
                    if w in clqdict.keys():
                        clqdict[w].insert(0, year)
                    else:
                        clqdict[w] = [year]

## Convert dictionary to json file

In [9]:
with open(jsonfile2, "w") as jf:
    jf.write("var words = [\n")
    jf.write("\t{text: 'colloquium', size: 102, yearlist: '1969-2018'},\n")
    for k, v in clqdict.items():
        jf.write("".join(("\t{text: '", k, "', size: ", str(len(v)), ", yearlist: ", str(v),"},\n")))
    jf.write("]")
print("Written file {}".format(jsonfile2))

Written file /home/ctroupin/ULg/gher-ulg.github.io/data/topicsWordle.js
